# Analyse des Essais Invalidés (Reset Edges_RoueAR)

**Objectif** : 
1. Détecter le reset (faux départ) dans TDMS.
2. Comparer les durées (Xsens vs TDMS Valide).
3. **Afficher les Start Times** pour synchro.

**Fichiers Cibles** : `Moto_Chicane_mouille_80`

In [ ]:
import numpy as np
import pandas as pd
from nptdms import TdmsFile
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

# Chemins ou Fichiers
TDMS_PATH = r'Moto_Chicane_mouille_80.tdms'
TXT_PATH = r'Moto_Chicane_mouille_80_P1.txt'
GROUP_NAME = 'P1'


In [ ]:
# --- CHARGEMENT XSENS ---
def load_xsens(path):
    print(f"--- Chargement Xsens : {path} ---")
    h_idx = None
    try:
        with open(path, 'r', errors='ignore') as f:
            for i, l in enumerate(f):
                if l.strip().startswith('PacketCounter'): h_idx=i; break
        if h_idx is None:
             with open(path, 'r', errors='ignore') as f:
                for i, l in enumerate(f): 
                    if 'UTC_Year' in l: h_idx=i; break
        if h_idx is None: raise ValueError("Header introuvable")
        
        try: df = pd.read_csv(path, sep='\t', header=h_idx)
        except: df = pd.read_csv(path, sep=r'\s+', header=h_idx)
        
        df.columns = df.columns.str.strip()
        
        # FIX Ghost Packets
        check_cols = [c for c in ['Acc_X', 'FreeAcc_E', 'Gyr_X'] if c in df.columns]
        if check_cols:
            df.dropna(subset=check_cols, how='all', inplace=True)
        
        req = ['UTC_Year', 'UTC_Month', 'UTC_Day', 'UTC_Hour', 'UTC_Minute', 'UTC_Second', 'UTC_Nano']
        if set(req).issubset(df.columns):
            df.dropna(subset=req, inplace=True)
            ts = pd.to_datetime(df[req[:-1]].astype(int).rename(columns={'UTC_Year':'year','UTC_Month':'month','UTC_Day':'day','UTC_Hour':'hour','UTC_Minute':'minute','UTC_Second':'second'}))
            df['TS_UTC'] = ts + pd.to_timedelta(df['UTC_Nano'], unit='ns')
            df.drop(columns=req, inplace=True)
            df.sort_values('TS_UTC', inplace=True)
            df.drop_duplicates(subset=['TS_UTC'], inplace=True)
            return df
        else: raise ValueError("Colonnes UTC manquantes")
    except Exception as e: print(f"Erreur Xsens: {e}"); return pd.DataFrame()

df_xs = load_xsens(TXT_PATH)
xsens_start = pd.NaT

if not df_xs.empty:
    xsens_start = df_xs['TS_UTC'].min()
else:
    print("[XSENS] Erreur chargement.")

In [ ]:
# ANALYSE TDMS & SYNTHESE
reset_index = None
tdms_start_time = None
tdms_fs = 400.0

try:
    tdms = TdmsFile.read(TDMS_PATH)
    group = tdms[GROUP_NAME]
    
    # 1. Recherche Timestamp Metadata (Méthode Specifique Channel Edges_RoueAR)
    # "start_time = pd.to_datetime(channel.properties.get('wf_start_time')).tz_localize(None)"
    
    if 'Edges_RoueAR' in group:
        chan = group['Edges_RoueAR']
        if 'wf_start_time' in chan.properties:
            raw_time = chan.properties['wf_start_time']
            tdms_start_time = pd.to_datetime(raw_time).tz_localize(None)
            print(f"  -> Trouvé (Channel 'Edges_RoueAR') : {tdms_start_time}")
            
    # Fallback Root si pas trouvé dans le channel
    if not tdms_start_time and 'wf_start_time' in tdms.properties:
         v = tdms.properties['wf_start_time']
         tdms_start_time = pd.to_datetime(v).tz_localize(None)
         print(f"  -> Trouvé (Root) : {tdms_start_time}")

    
    # 2. Data & Reset
    chan_edges = group['Edges_RoueAR']
    edges_data = chan_edges[:]
    
    def detect_reset_index(data, threshold=-100):
        diffs = np.diff(data)
        resets = np.where(diffs < threshold)[0]
        if len(resets) > 0: return resets[0] + 1
        return None

    reset_index = detect_reset_index(edges_data)
    
    # --- AFFICHAGE RESULTATS ---
    print("\n================ SYNTHESE ================")
    print(f"1. XSENS START Time       : {xsens_start}")
    
    if tdms_start_time:
        print(f"2. TDMS START Time (Meta) : {tdms_start_time}")
    else:
        print(f"2. TDMS START Time (Meta) : [NON TROUVE]")
        
    print(f"\n3. DETECTION RESET")
    if reset_index:
        print(f"   -> Reset détecté à l'index : {reset_index}")
        dur_invalid = reset_index / tdms_fs
        print(f"   -> Durée partie invalide   : {dur_invalid:.2f} s")
        
        if tdms_start_time:
             tdms_start_valid = tdms_start_time + timedelta(seconds=dur_invalid)
             print(f"   -> TDMS Start Partie 2     : {tdms_start_valid} (Calculé)")
             
             delta = xsens_start - tdms_start_valid
             print(f"   -> DECALAGE (Xsens - Valid): {delta.total_seconds():.4f} s")
    else:
        print("   -> Aucun reset détecté.")
        
    print(f"\n4. COMPARAISON TAILLES")
    xsens_count = len(df_xs)
    tdms_valid_count = len(edges_data) - (reset_index if reset_index else 0)
    print(f"   Xsens Total : {xsens_count}")
    print(f"   TDMS Valide : {tdms_valid_count}")
    print(f"   Difference  : {xsens_count - tdms_valid_count}")
    
except Exception as e:
    print(f"Erreur: {e}")